# Table of Contents
 <p>

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# display full, non-truncated rows
pd.set_option('display.max_colwidth', None)

# columns = ['class', 'unsure', 'area_code?',
#            'date', 'camera', 'satellite',
#           'deforested_area', 'unknown',
#             'area', 'state', 'abbreviation',
#           'lat', 'long']

brasil_data = pd.read_csv('deter_public_amz_deter_public.csv')

In [3]:
brasil_data

,CLASSNAME,QUADRANT,PATH_ROW,VIEW_DATE,SENSOR,SATELLITE,AREAUCKM,UC,AREAMUNKM,MUNICIPALI,UF,X,Y
0,CICATRIZ_DE_QUEIMADA,NaN,165105,7/27/2020,AWFI,CBERS-4,0.000000,NaN,0.075231,Novo Repartimento,PA,-50.4712,-4.91754
1,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,0.344234,Altamira,PA,-55.0566,-6.91208
2,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,4.788569,Brasil Novo,PA,-52.5766,-3.56411
3,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,1.067603,Altamira,PA,-55.2042,-6.78340
4,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,0.067680,Alenquer,PA,-54.9766,-1.79718
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186408,MINERACAO,NaN,210124,6/24/2020,WFI,CBERS-4A,0.000000,NaN,0.110526,Cumaru do Norte,PA,-50.9172,-7.76491
186409,MINERACAO,NaN,165105,7/27/2020,AWFI,CBERS-4,0.000000,NaN,0.095688,Bannach,PA,-50.8812,-7.41031
186410,MINERACAO,NaN,165105,7/27/2020,AWFI,CBERS-4,0.000000,NaN,0.063135,Bannach,PA,-50.8652,-7.38636
186411,MINERACAO,NaN,169105,1/14/2018,AWFI,CBERS-4,0.121459,├üREA DE PROTE├ç├âO AMBIENTAL DO TAPAJ├ôS,0.121459,Jacareacanga,PA,-57.4651,-6.16502


In [4]:
print(brasil_data['X'][0])
print(brasil_data['Y'][0])

-50.4712
-4.91754


In [5]:
brasil_data.reset_index(drop=False, inplace=True)

In [6]:
brasil_data['base_url'] = brasil_data.apply(lambda row: f"https://www.planet.com/explorer/#/center/{row['X']:.8},{row['Y']:.8}/zoom/8", axis=1)

In [7]:
brasil_data['base_url']

0         https://www.planet.com/explorer/#/center/-50.4712,-4.91754/zoom/8
1         https://www.planet.com/explorer/#/center/-55.0566,-6.91208/zoom/8
2         https://www.planet.com/explorer/#/center/-52.5766,-3.56411/zoom/8
3          https://www.planet.com/explorer/#/center/-55.2042,-6.7834/zoom/8
4         https://www.planet.com/explorer/#/center/-54.9766,-1.79718/zoom/8
                                        ...                                
186408    https://www.planet.com/explorer/#/center/-50.9172,-7.76491/zoom/8
186409    https://www.planet.com/explorer/#/center/-50.8812,-7.41031/zoom/8
186410    https://www.planet.com/explorer/#/center/-50.8652,-7.38636/zoom/8
186411    https://www.planet.com/explorer/#/center/-57.4651,-6.16502/zoom/8
186412    https://www.planet.com/explorer/#/center/-57.3608,-5.73095/zoom/8
Name: base_url, Length: 186413, dtype: object

In [8]:
brasil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186413 entries, 0 to 186412
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   index       186413 non-null  int64  
 1   CLASSNAME   186413 non-null  object 
 2   QUADRANT    49516 non-null   object 
 3   PATH_ROW    186413 non-null  int64  
 4   VIEW_DATE   186413 non-null  object 
 5   SENSOR      186413 non-null  object 
 6   SATELLITE   186413 non-null  object 
 7   AREAUCKM    186413 non-null  float64
 8   UC          8568 non-null    object 
 9   AREAMUNKM   186413 non-null  float64
 10  MUNICIPALI  186411 non-null  object 
 11  UF          186411 non-null  object 
 12  X           186413 non-null  float64
 13  Y           186413 non-null  float64
 14  base_url    186413 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 21.3+ MB


In [9]:
geometry = [Point(xy) for xy in zip(brasil_data.X, brasil_data.Y)]

In [10]:
brasil_data_gdf = gpd.GeoDataFrame(brasil_data, geometry=geometry)

In [11]:
# brasil_data_gdf.crs = 'epsg:4674'

In [12]:
brasil_data_gdf.crs = 'epsg:4326'

cntry_crs = 32721

brasil_data_gdf_m = brasil_data_gdf.to_crs('epsg:{}'.format(cntry_crs))

In [13]:
deg_to_meters_lat_minus_seven = 110590
five_km_in_deg = 5000/110590.
def create_buffer(row):
    dist = five_km_in_deg
    point = row["geometry"]
    row["geometry"] = point.buffer(dist)
    return row

In [14]:
brasil_data_buffer_gdf = brasil_data_gdf.apply(create_buffer,axis=1)

In [15]:
brasil_data_buffer_gdf.crs = 'epsg:4326'

In [17]:
brasil_data_buffer_gdf.to_file('test_4326.shp')

In [34]:
brasil_data_buffer_gdf['wkt'] = brasil_data_buffer_gdf.apply(lambda row: row.geometry.simplify(0.0005).wkt.replace(' ',''), axis=1)

In [38]:
brasil_data_buffer_gdf.drop(columns='geometry').to_csv('deter_public_amz_deter_public_extended.csv')